# Benchmark


**!** This notebook has to be run after the following notebooks:

-   [dataset.ipynb](dataset.ipynb)
-   [scoring.ipynb](scoring.ipynb)
-   [model.ipynb](model.ipynb)


In [59]:
import model
import dataset
import scoring

from typing import *

In [60]:
models: Dict[str, model.Tracker] = model.load_models()

| using constant padding
| using scales: [0.8333333333333334, 1.0, 1.2]
| using ordinary correlation
load pretrained model from models/SiamSE/checkpoint_vot.pth
remove prefix "module."
missing keys:set()
unused checkpoint keys:set()
| using constant padding
| using scales: [0.8333333333333334, 1.0, 1.2]
| using ordinary correlation
load pretrained model from models/SiamSE/checkpoint_vot.pth
remove prefix "module."
missing keys:set()
unused checkpoint keys:set()


/home/leiyks/git/DNN/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leiyks/git/DNN/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [61]:
datasets: Dict = dataset.load_datasets()

Error while loading dataset /home/leiyks/git/DNN/data/mytc128/David gt and image files have different length
Error while loading dataset /home/leiyks/git/DNN/data/mytc128/Football1 gt and image files have different length
Error while loading dataset /home/leiyks/git/DNN/data/mytc128/CarScale gt and image files have different length
Error while loading dataset /home/leiyks/git/DNN/data/mytc128/Jogging2 gt and image files have different length
Error while loading dataset /home/leiyks/git/DNN/data/mytc128/Jogging1 gt and image files have different length
Error while loading dataset /home/leiyks/git/DNN/data/mytc128/Subway gt and image files have different length


In [62]:
models.keys(), datasets.keys()

(dict_keys(['SEsiamFC', 'AAA', 'PyECO']), dict_keys(['mytc128', 'myvot2021']))

# Tracking


In [63]:
import cv2
import numpy as np
import json
import os

from pathlib import Path

In [64]:
def write_video(out, image_file, gt, pred=None):
    """Write a frame to the video.

    Args:
        out (cv2.VideoWriter): Video writer.
        image_file (str): Path to the image file.
        gt (List[int]): Ground truth bounding box.
        pred (List[int], optional): Predicted bounding box. Defaults to None.
    """
    im = cv2.imread(image_file)
    cv2.rectangle(
        im,
        (int(gt[0]), int(gt[1])),
        (int(gt[0] + gt[2]), int(gt[1] + gt[3])),
        (0, 255, 0),
        2,
    )
    if pred is not None:
        cv2.rectangle(
            im,
            (int(pred[0]), int(pred[1])),
            (int(pred[0] + pred[2]), int(pred[1] + pred[3])),
            (0, 0, 255),
            2,
        )
    out.write(im)


def track_video(tracker, video, verbose=0, save_video=None):
    """Track a whole video.

    Args:
        tracker (model.Tracker): Tracker.
        video (Dict): Video.
        verbose (int, optional): Verbosity level. Defaults to 0.
        save_video (str, optional): Path to the video to save. Defaults to None.

    Returns:
        Tuple[float, float, float]: Precision, Success, FPS.
    """
    start_frame, toc = 0, 0

    pred = []
    image_files, gt = video["image_files"], video["gt"]

    if save_video is not None:
        sh = cv2.imread(image_files[0]).shape[:2]
        frame_size = (sh[1], sh[0])
        out = cv2.VideoWriter(
            save_video, cv2.VideoWriter_fourcc(*"DIVX"), 20.0, frame_size
        )

    for f, image_file in enumerate(image_files):
        tic = cv2.getTickCount()

        if f == start_frame:  # init
            tracker.initialize(image_file, np.array(gt[f]))
            pred.append(gt[f])

            if save_video is not None:
                write_video(out, image_file, gt[f])

        elif f > start_frame:  # tracking
            pred_bbox = tracker.track(image_file)
            b_overlap = scoring.get_precision(gt[f], pred_bbox)
            if b_overlap > 0:
                pred.append(pred_bbox)
            else:
                pred.append(2)
                start_frame = f + 5

            if verbose > 1:
                print(f"{f} gt: {gt[f]} pred: {pred_bbox} overlap: {b_overlap}")

            if save_video is not None:
                write_video(out, image_file, gt[f], pred_bbox)
        else:
            pred.append(0)
            if save_video is not None:
                write_video(out, image_file, gt[f])

        toc += cv2.getTickCount() - tic

    toc /= cv2.getTickFrequency()

    precisions = [scoring.get_precision(gt[i], pred[i]) for i in range(len(gt))]
    precisions = np.array(precisions)
    mprec = np.mean(precisions)

    success = [scoring.is_success(gt[i], pred[i]) for i in range(len(gt))]
    success = np.array(success)
    msucc = np.mean(success)

    fps = f / toc

    if verbose > 0:
        print(
            f'Video: {video["name"]:12s} Time: {toc:2.1f}s Speed: {fps:5.2f}fps mSuccess: {msucc:3.2f} mPrecision {mprec:3.2f}'
        )

    if save_video is not None:
        out.release()

    return mprec, msucc, fps

In [65]:
dataset = "mytc128"
tracker = "AAA"

In [66]:
datasets[dataset].keys()

dict_keys(['TennisBall_ce', 'Torus', 'Yo-yos_ce1', 'Spiderman_ce', 'Eagle_ce', 'Logo_ce', 'Couple', 'Fish_ce2', 'Basketball', 'Baby_ce', 'Ball_ce3', 'Surf_ce4', 'Skating_ce2', 'Busstation_ce1', 'Cup_ce', 'Deer', 'Yo-yos_ce2', 'Pool_ce2', 'Face_ce2', 'Walking2', 'TableTennis_ce', 'MountainBike', 'Microphone_ce1', 'Fish_ce1', 'Girlmov', 'Pool_ce1', 'Singer1', 'Plate_ce1', 'Sailor_ce', 'Ironman', 'SuperMario_ce', 'Surf_ce2', 'Woman', 'Bolt', 'Bike_ce1', 'Bikeshow_ce', 'Walking', 'Ball_ce1', 'Shaking', 'Ball_ce2', 'Basketball_ce1', 'Basketball_ce2', 'Carchasing_ce1', 'Railwaystation_ce', 'Messi_ce', 'Surf_ce3', 'Ball_ce4', 'Matrix', 'Biker', 'Bee_ce', 'Iceskater', 'Guitar_ce2', 'Skiing', 'Toyplane_ce', 'Plate_ce2', 'Boat_ce1', 'Kite_ce3', 'Surf_ce1', 'Electricalbike_ce', 'Juice', 'FaceOcc1', 'Badminton_ce1', 'Boat_ce2', 'MotorRolling', 'Tennis_ce3', 'Panda', 'Sunshade', 'Hurdle_ce2', 'Singer2', 'Plane_ce2', 'Bicycle', 'Motorbike_ce', 'Airport_ce', 'Liquor', 'Doll', 'Tiger2', 'Boy', 'Hurdle

In [67]:
video = "Tiger1"
t1 = models[tracker]
v1 = datasets[dataset][video]

video_name = f"{dataset}_{tracker}_{video}.avi"
track_video(t1, v1, verbose=1, save_video=video_name)

Video: Tiger1       Time: 30.3s Speed: 11.65fps mSuccess: 0.60 mPrecision 0.50


(0.5020210933957814, 0.5988700564971752, 11.65355821343052)

In [68]:
from ipywidgets import Video, Image

new_ = Video.from_file(video_name, play=True, width=360, height=360)
new_

Video(value=b'RIFF\xeeC}\x00AVI LIST\xec\x11\x00\x00hdrlavih8\x00\x00\x00P\xc3\x00\x00\x00p\x17\x00...', forma…

In [72]:
def track_dataset(
    tracker, dataset, dataset_name, n=3, verbose=0, save_results=False
) -> Tuple[float, float, float]:
    """Track a whole dataset.

    Args:
        tracker (model.Tracker): Tracker.
        dataset (Dict): Dataset.
        dataset_name (str): Dataset name.
        n (int, optional): Number of videos to track. Defaults to 3.
        verbose (int, optional): Verbosity level. Defaults to 0.
        save_results (bool, optional): Save results. Defaults to False.

    Returns:
        Tuple[float, float, float]: Mean Precisions, Successes, FPSs.
    """
    precisions: List[float] = []
    success: List[float] = []
    fpss: List[float] = []

    result_folder = Path("results")
    if save_results:
        (result_folder / tracker.model_name).mkdir(parents=True, exist_ok=True)

    results: Dict = {}
    count = 0

    for video in dataset:
        if count == n:
            break

        precision, succes, fps = track_video(tracker, dataset[video], verbose)

        precisions.append(precision)
        success.append(succes)
        fpss.append(fps)

        if save_results:
            results[video] = {"precision": precision, "succes": succes, "fps": fps}

        count += 1

    if save_results:
        with open(result_folder / tracker.model_name / (dataset_name + ".json"), 'w') as f:
            json.dump(results, f, indent=2)

    return np.mean(precisions), np.mean(success), np.mean(fpss)

In [73]:
def do_benchmark(models, datasets, verbose=0, save_results=False):
    """Benchmark a set of models on a set of datasets.

    Args:
        models (Dict): Models.
        datasets (Dict): Datasets.
        verbose (int, optional): Verbosity level. Defaults to 0.
        save_results (bool, optional): Save results. Defaults to False.
    """
    for model_name in models:
        for dataset_name in datasets:
            print(f"Benchmarking {model_name} on {dataset_name}")
            track_dataset(
                models[model_name],
                datasets[dataset_name],
                dataset_name,
                verbose=verbose,
                save_results=save_results,
            )

In [74]:
do_benchmark(models, datasets, verbose=1, save_results=True)

Benchmarking SEsiamFC on mytc128
Video: TennisBall_ce Time: 7.8s Speed: 36.79fps mSuccess: 0.53 mPrecision 0.38
Video: Torus        Time: 10.4s Speed: 25.27fps mSuccess: 0.99 mPrecision 0.77
Video: Yo-yos_ce1   Time: 6.7s Speed: 34.78fps mSuccess: 0.74 mPrecision 0.56
Benchmarking SEsiamFC on myvot2021
Benchmarking AAA on mytc128


/home/leiyks/git/DNN/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Video: TennisBall_ce Time: 18.2s Speed: 15.74fps mSuccess: 0.55 mPrecision 0.40
Video: Torus        Time: 23.3s Speed: 11.27fps mSuccess: 1.00 mPrecision 0.80
Video: Yo-yos_ce1   Time: 13.5s Speed: 17.32fps mSuccess: 0.71 mPrecision 0.55
Benchmarking AAA on myvot2021
Benchmarking PyECO on mytc128


/home/leiyks/git/DNN/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Video: TennisBall_ce Time: 9.0s Speed: 32.03fps mSuccess: 0.53 mPrecision 0.41
Video: Torus        Time: 4.7s Speed: 56.19fps mSuccess: 1.00 mPrecision 0.84
Video: Yo-yos_ce1   Time: 4.1s Speed: 56.95fps mSuccess: 0.73 mPrecision 0.56
Benchmarking PyECO on myvot2021


/home/leiyks/git/DNN/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
